In [ ]:
import numpy as np
import pandas as pd
from typing import Tuple
import matplotlib.pyplot as plt

#### Word2Vec
import gensim
from gensim.models import Word2Vec

####
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import torch.nn as nn
import torchutils as tu
from sklearn.model_selection import train_test_split
from collections import Counter

from torchmetrics.classification import BinaryAccuracy
from src.rnn_preprocessing import (
                                data_preprocessing, 
                                preprocess_single_string, 
                                padding, 
                                get_words_by_freq
                                )
from src.train_rnn import train_attention_lstm

In [ ]:
# Лучше всего установить такую же версию
print(gensim.__version__)